In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import bipartite
import copy
from sklearn.preprocessing import StandardScaler
import scipy.special as special
import scipy.sparse as sp


import os
import statsmodels.api as sm

import tensorflow as tf
#import tensorflow_addons as tfa
import keras

from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, GRU, Input
from tensorflow.keras.optimizers import SGD, Adam


from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, LeakyReLU, GRU, Concatenate, Reshape, Softmax, Attention
from tensorflow.keras import activations
from tensorflow.keras.models import Model
#from keras.layers import LeakyReLU

from spektral.layers import  GCSConv, DiffusionConv, GATConv, ARMAConv, GCNConv

from tensorflow.keras.regularizers import l2



In [3]:
FF4_Data = pd.read_csv('Beta_suit_monthly.csv')
FF4_Data['Year'] = FF4_Data['DATE'].astype(str).str[:4].astype(int)
FF4_Data['Month'] = FF4_Data['DATE'].astype(str).str[4:6].astype(int)
# remove % and convert excess return to values
FF4_Data['exret'] = FF4_Data['exret'].str.rstrip('%').astype('float') / 100.0

FF4_Data = FF4_Data[['Year', 'Month', 'PERMNO','b_mkt', 'b_smb', 'b_hml',  'b_umd', 'R2', 'exret']]
FF4_Data.sort_values(by=['PERMNO', 'Year', 'Month'], inplace = True)

CRSP_data =  pd.read_pickle('/largedisks/ajim/KDD21_data/CRSP_data.pickle')
CRSP_data['Year'] =  pd.DatetimeIndex(CRSP_data['date']).year
CRSP_data['Month'] =  pd.DatetimeIndex(CRSP_data['date']).month


In [361]:
CRSP_data

,CUSIP,PERMNO,date,TICKER,ret,me,m_1,m_6,m_12,size,...,b_mkt,b_smb,b_hml,ivol,tvol,RF,ex_ret,ID,Year,Month
1484931,00036110,54594,1990-01-31,AIR,-0.121667,5.042835e+05,0.014085,0.062731,0.280000,13.130894,...,1.2396,0.3213,-0.7656,0.052705,0.096792,0.0057,-0.127367,0,1990,1
584632,00282410,20482,1990-01-31,ABT,-0.038971,1.463462e+07,-0.018051,0.034221,0.387755,16.498901,...,0.7767,-0.7074,-1.1689,0.048739,0.076934,0.0057,-0.044671,1,1990,1
1991499,00365410,75107,1990-01-31,ABD,-0.170455,3.397238e+04,-0.083333,-0.128713,0.000000,10.433303,...,1.2246,2.1519,0.9133,0.107950,0.145632,0.0057,-0.176155,2,1990,1
1637709,00481610,60038,1990-01-31,ACU,0.030303,1.348525e+04,-0.070423,-0.164557,-0.340000,9.509352,...,0.4988,0.9817,0.3892,0.077236,0.084885,0.0057,0.024603,3,1990,1
2049332,00724F10,75510,1990-01-31,ADBE,0.162963,4.724440e+05,0.051948,-0.109890,-0.124324,13.065675,...,0.4518,2.2542,-1.7090,0.163018,0.185374,0.0057,0.157263,4,1990,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2739589,96990410,83011,2019-12-31,WSM,0.058213,5.689176e+06,0.039078,0.067692,0.375619,15.554076,...,0.6103,1.7060,-0.2359,0.053407,0.074848,0.0014,0.056813,438,2019,12
1381861,97463710,51086,2019-12-31,WGO,0.114664,1.784366e+06,-0.011234,0.229754,0.963238,14.394574,...,1.3376,0.9051,1.6828,0.105293,0.130394,0.0014,0.113264,439,2019,12
1123789,97809710,42286,2019-12-31,WWW,0.054206,2.728756e+06,0.081536,0.165577,0.006585,14.819356,...,0.9802,0.7889,-0.0533,0.061021,0.075962,0.0014,0.052806,440,2019,12
2783719,98181110,83601,2019-12-31,WOR,0.107572,2.323865e+06,0.040478,-0.048684,0.099311,14.658742,...,1.2794,0.4024,0.8228,0.068919,0.088417,0.0014,0.106172,441,2019,12


In [8]:
CRSP_just_ret_data =  pd.read_pickle('/largedisks/ajim/KDD21_data/Just_ret_data_test_result.pickle')
CRSP_just_ret_data = CRSP_just_ret_data.stack().reset_index()
CRSP_just_ret_data.rename(columns={'level_1': "ID", 0: 'DY_Pred'}, inplace =True) 

In [9]:
CRSP_just_ret_data

,date,ID,DY_Pred
0,2014-01-31,0,0.045970
1,2014-01-31,1,0.020188
2,2014-01-31,2,0.035505
3,2014-01-31,3,0.012921
4,2014-01-31,4,0.031787
...,...,...,...
31891,2019-12-31,438,0.011242
31892,2019-12-31,439,0.008415
31893,2019-12-31,440,-0.005787
31894,2019-12-31,441,0.015162


In [17]:
CRSP_data_2

,CUSIP,PERMNO,date,TICKER,ret,me,m_1,m_6,m_12,size,...,b_smb,b_hml,ivol,tvol,RF,ex_ret,ID,Year,Month,DY_Pred
0,00036110,54594,2014-01-31,AIR,-0.045876,1055340.00,-0.102819,0.155528,0.485942,13.869374,...,1.7755,0.0062,0.077856,0.116344,0.0000,-0.045876,0,2014,1,0.045970
1,00282410,20482,2014-01-31,ABT,-0.037829,56683178.76,0.003666,0.046410,0.131346,17.852988,...,-1.1287,-0.0605,0.033189,0.044699,0.0000,-0.037829,1,2014,1,0.020188
2,00365410,75107,2014-01-31,ABMD,0.028048,1094596.82,-0.064708,0.066188,0.916846,13.905897,...,0.4100,-0.6972,0.132741,0.151227,0.0000,0.028048,2,2014,1,0.035505
3,00481610,60038,2014-01-31,ACU,-0.011409,46821.40,0.010855,0.073487,0.221311,10.754096,...,-0.1289,0.4176,0.042482,0.044985,0.0000,-0.011409,3,2014,1,0.012921
4,00724F10,75510,2014-01-31,ADBE,-0.011507,29393990.76,0.054579,0.266476,0.582844,17.196301,...,-0.2591,0.6805,0.052144,0.071444,0.0000,-0.011507,4,2014,1,0.031787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31891,96990410,83011,2019-12-31,WSM,0.058213,5689176.48,0.039078,0.067692,0.375619,15.554076,...,1.7060,-0.2359,0.053407,0.074848,0.0014,0.056813,438,2019,12,0.011242
31892,97463710,51086,2019-12-31,WGO,0.114664,1784366.40,-0.011234,0.229754,0.963238,14.394574,...,0.9051,1.6828,0.105293,0.130394,0.0014,0.113264,439,2019,12,0.008415
31893,97809710,42286,2019-12-31,WWW,0.054206,2728756.24,0.081536,0.165577,0.006585,14.819356,...,0.7889,-0.0533,0.061021,0.075962,0.0014,0.052806,440,2019,12,-0.005787
31894,98181110,83601,2019-12-31,WOR,0.107572,2323864.92,0.040478,-0.048684,0.099311,14.658742,...,0.4024,0.8228,0.068919,0.088417,0.0014,0.106172,441,2019,12,0.015162


In [ ]:
CRSP_data_2 = CRSP_data.merge(CRSP_just_ret_data, on = ['date', 'ID'], how =  'inner')
FF4_Data_2 = FF4_Data.merge(CRSP_data_2[['ex_ret','ID','date', 'Year','Month',  'DY_Pred', 'PERMNO']], on= ['Year','Month', 'PERMNO'], how =  'inner')

FF4_Data_2.sort_values(by=["Year","date"], inplace =True)
FF4_Data_2.dropna(inplace =True)

FF4_Data_2.set_index(['ID', 'date'], inplace = True)



In [106]:
exog = sm.add_constant(FF4_Data_2[['b_mkt','b_smb','b_hml','b_umd', 'DY_Pred']])
endog = FF4_Data_2[['ex_ret']]

mod = sm.OLS(endog, exog)
results = mod.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ex_ret   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     20.42
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           2.04e-20
Time:                        20:26:38   Log-Likelihood:                 30829.
No. Observations:               31893   AIC:                        -6.165e+04
Df Residuals:                   31887   BIC:                        -6.160e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.001      6.213      0.000       0.005       0.009
b_mkt         -0.0008      0.001     -0.773      0.439      -0.003       0.001
b_smb         -0.0005      0.001     -0.649      0.516      -0.002       0.001
b_hml          0.0001      0.001      0.146      0.884      -0.002       0.002
b_umd         -0.0041      0.001     -3.742      0.000      -0.006      -0.002
DY_Pred       -0.1444      0.015     -9.477      0.000      -0.174      -0.115
==============================================================================
Omnibus:                    18190.152   Durbin-Watson:                   1.620
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2136869.215
Skew:                           1.795   Prob(JB):                         0.00
Kurtosis:                      42.939   Cond. No.                         45.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [364]:
# centrality breakdown
grp_cen = FF4_Data_2.groupby(['date'])['DY_Pred'].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]).reset_index()

In [365]:
grp_cen = grp_cen[["date", '10%', '20%', '30%', '40%','50%', '60%', '70%', '80%', '90%']]

grp_cen = grp_cen.rename(columns={'10%':'cn10','20%':'cn20','30%':'cn30','40%':'cn40',
                       '50%':'cn50','60%':'cn60','70%':'cn70', '80%':'cn80',
                        '90%':'cn90'})

In [366]:
centrality_grp =  FF4_Data_2[['ID','date', 'exret','ex_ret', 'DY_Pred']].merge(grp_cen, how='left', on=['date'])

In [367]:
def cn_bucket(row):
    if row['DY_Pred']<=row['cn10']:
        value = 'CD10'
    elif row['DY_Pred']<=row['cn20']:
        value='CD20'
    elif row['DY_Pred']<=row['cn30']:
        value='CD30'
    elif row['DY_Pred']<=row['cn40']:
        value='CD40'   
    elif row['DY_Pred']<=row['cn50']:
        value='CD50'     
    elif row['DY_Pred']<=row['cn60']:
        value='CD60' 
    elif row['DY_Pred']<=row['cn70']:
        value='CD70'         
    elif row['DY_Pred']<=row['cn80']:
        value='CD80'             
    elif row['DY_Pred']<=row['cn90']:
        value='CD90'
    elif row['DY_Pred']>row['cn90']:
        value='CD100'  
        
    else:
        value=''
    return value

In [368]:
centrality_grp['cn_port'] = centrality_grp.apply(cn_bucket, axis=1)

In [369]:
centrality_grp

,ID,date,exret,ex_ret,DY_Pred,cn10,cn20,cn30,cn40,cn50,cn60,cn70,cn80,cn90,cn_port
0,207,2014-01-31,0.001111,-0.005531,-0.016035,-0.008077,0.005342,0.011342,0.017756,0.023180,0.027110,0.030658,0.037156,0.047132,CD10
1,315,2014-01-31,-0.069292,-0.096789,0.039958,-0.008077,0.005342,0.011342,0.017756,0.023180,0.027110,0.030658,0.037156,0.047132,CD90
2,294,2014-01-31,-0.002068,-0.032410,0.029587,-0.008077,0.005342,0.011342,0.017756,0.023180,0.027110,0.030658,0.037156,0.047132,CD70
3,7,2014-01-31,-0.093430,-0.077985,-0.013540,-0.008077,0.005342,0.011342,0.017756,0.023180,0.027110,0.030658,0.037156,0.047132,CD10
4,374,2014-01-31,0.078692,0.044961,0.012519,-0.008077,0.005342,0.011342,0.017756,0.023180,0.027110,0.030658,0.037156,0.047132,CD40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31888,186,2019-12-31,-0.062569,-0.042666,-0.008600,-0.005337,0.004912,0.010173,0.015329,0.020934,0.025556,0.032090,0.038886,0.049118,CD10
31889,223,2019-12-31,0.060725,0.112781,0.008815,-0.005337,0.004912,0.010173,0.015329,0.020934,0.025556,0.032090,0.038886,0.049118,CD30
31890,254,2019-12-31,0.002601,0.047346,0.033930,-0.005337,0.004912,0.010173,0.015329,0.020934,0.025556,0.032090,0.038886,0.049118,CD80
31891,295,2019-12-31,-0.031845,0.044919,0.019224,-0.005337,0.004912,0.010173,0.015329,0.020934,0.025556,0.032090,0.038886,0.049118,CD50


In [425]:
FF = centrality_grp.groupby(['date', 'cn_port'])['ex_ret'].mean().reset_index()
FF.groupby(['cn_port'])['ex_ret'].mean().reset_index()

,cn_port,ex_ret
0,CD10,0.011025
1,CD100,0.006822
2,CD20,0.008390
3,CD30,0.009581
4,CD40,0.008815
5,CD50,0.009987
6,CD60,0.006786
7,CD70,0.010854
8,CD80,0.008598
9,CD90,0.006697


In [428]:
(FF.groupby(['cn_port'])['ex_ret'].mean()/FF.groupby(['cn_port'])['ex_ret'].sem()).reset_index()

,cn_port,ex_ret
0,CD10,2.282620
1,CD100,1.243447
2,CD20,1.630009
3,CD30,2.134351
4,CD40,1.752471
5,CD50,1.947299
6,CD60,1.456343
7,CD70,2.123813
8,CD80,1.804799
9,CD90,1.467517


In [430]:
CR_10 = FF[(FF['cn_port']=='CD90')]
CR_100 = FF[(FF['cn_port']=='CD10')]
print(np.round(CR_100['ex_ret'].mean()-CR_10['ex_ret'].mean(), 4))

print(stats.ttest_ind(CR_100['ex_ret'], CR_10['ex_ret']))

0.0043
Ttest_indResult(statistic=0.6513393942917206, pvalue=0.5158801292068775)


In [431]:
0.0043*12*100

5.16

In [432]:
43*12

516

In [ ]:
#FF4_Data_2.reset_index(inplace = True)

In [155]:
result = fama_macbeth(FF4_Data_2,'date','exret',['b_mkt','b_smb','b_hml', 'b_umd', 'DY_Pred'],intercept=True)
result.head()
fm_summary(result)

,mean,std_error,tstat
intercept,0.008217,0.002407,3.412942
b_mkt,-0.009427,0.002061,-4.575026
b_smb,0.002956,0.001450,2.039076
b_hml,0.001191,0.001662,0.716562
b_umd,-0.005001,0.002454,-2.037767
DY_Pred,-0.024043,0.037178,-0.646710


In [164]:
F_NLTC_Summary

,mean,std_error,tstat
Intercept,0.008716,0.002149,4.056106
b_mkt,-0.000678,0.004368,-0.155162
b_smb,0.000754,0.003099,0.243123
b_hml,-0.002252,0.003441,-0.654556
b_umd,-0.002442,0.004750,-0.514125
DY_Pred,-0.019147,0.034341,-0.557568


In [147]:
import statsmodels.api as sm

import statsmodels.formula.api as smf

In [148]:
# For Fama Mackbath Model

def ols_coef(x,formula):
    return smf.ols(formula,data=x).fit().params


def fm_summary(p):
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std_error','tstat']]

In [161]:
Fama_NLTC_MP = (FF4_Data_2.groupby(['date']).apply(ols_coef,'ex_ret ~ 1 +  b_mkt + b_smb + b_hml + b_umd + DY_Pred'))

In [162]:
F_NLTC_Summary = fm_summary(Fama_NLTC_MP)

In [165]:
Fama_NLTC_MP 

,Intercept,b_mkt,b_smb,b_hml,b_umd,DY_Pred
date,,,,,,
2014-01-31,0.024998,-0.050287,-0.008780,-0.002409,-0.019416,-0.182158
2014-02-28,0.006445,0.040301,-0.002410,0.003570,-0.008157,-0.008989
2014-03-31,0.018587,-0.015473,-0.001621,0.021321,0.011147,-0.208387
2014-04-30,0.007847,-0.004101,-0.028191,-0.003670,-0.014381,-0.197048
2014-05-30,-0.003157,0.016034,-0.015124,-0.003209,-0.005680,-0.258264
...,...,...,...,...,...,...
2019-08-30,0.020588,-0.033595,-0.026203,-0.052428,0.071586,-0.671013
2019-09-30,0.014325,0.014482,0.003382,0.049413,-0.046521,-0.008674
2019-10-31,0.007341,0.019362,0.030973,-0.023158,-0.009298,-0.689544


In [163]:
F_NLTC_Summary

,mean,std_error,tstat
Intercept,0.008716,0.002149,4.056106
b_mkt,-0.000678,0.004368,-0.155162
b_smb,0.000754,0.003099,0.243123
b_hml,-0.002252,0.003441,-0.654556
b_umd,-0.002442,0.004750,-0.514125
DY_Pred,-0.019147,0.034341,-0.557568


## Daily Data

In [198]:
daily_data =  pd.read_pickle('/home/ajim/Desktop/Graph Asset Pricing/Daily/daily_clear_ret.pickle')

In [199]:
just_ret = daily_data[['datadate', 'cusip', 'ret']]
just_ret['Year'] =  pd.DatetimeIndex(just_ret['datadate']).year
just_ret['Month'] =  pd.DatetimeIndex(just_ret['datadate']).month
just_ret['Day'] =  pd.DatetimeIndex(just_ret['datadate']).day

/tmp/ipykernel_157934/2798508316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_ret['Year'] =  pd.DatetimeIndex(just_ret['datadate']).year
/tmp/ipykernel_157934/2798508316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_ret['Month'] =  pd.DatetimeIndex(just_ret['datadate']).month
/tmp/ipykernel_157934/2798508316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [200]:
FF_4_daily = pd.read_csv('/home/ajim/Desktop/Graph Asset Pricing/Review Feb 2022/FF4_daily_Sep2022.csv')

In [201]:
FF_4_daily['Year'] = FF_4_daily['date'].astype(str).str[:4].astype(int)
FF_4_daily['Month'] = FF_4_daily['date'].astype(str).str[4:6].astype(int)
FF_4_daily['Day'] = FF_4_daily['date'].astype(str).str[6:].astype(int)

In [202]:
FF_4_daily

,date,mktrf,smb,hml,rf,umd,Year,Month,Day
0,19600104,-0.0003,0.0060,0.0070,0.00017,-0.0079,1960,1,4
1,19600105,0.0078,-0.0041,0.0054,0.00017,-0.0047,1960,1,5
2,19600106,-0.0047,0.0013,0.0038,0.00017,0.0007,1960,1,6
3,19600107,-0.0065,0.0037,0.0009,0.00017,-0.0029,1960,1,7
4,19600108,-0.0033,0.0018,0.0010,0.00017,-0.0043,1960,1,8
...,...,...,...,...,...,...,...,...,...
15769,20220825,0.0145,0.0018,-0.0001,0.00008,-0.0077,2022,8,25
15770,20220826,-0.0338,-0.0055,0.0169,0.00008,0.0195,2022,8,26
15771,20220829,-0.0072,-0.0056,0.0042,0.00008,0.0062,2022,8,29
15772,20220830,-0.0111,-0.0025,-0.0024,0.00008,-0.0086,2022,8,30


/tmp/ipykernel_157934/2205933811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_ret['Year'] =  pd.DatetimeIndex(just_ret['datadate']).year
/tmp/ipykernel_157934/2205933811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  just_ret['Month'] =  pd.DatetimeIndex(just_ret['datadate']).month
/tmp/ipykernel_157934/2205933811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [203]:
just_ret

,datadate,cusip,ret,Year,Month,Day
11155,2011-01-03,723484101,0.000724,2011,1,3
11156,2011-01-04,723484101,0.004330,2011,1,4
11157,2011-01-05,723484101,-0.006744,2011,1,5
11158,2011-01-06,723484101,0.001931,2011,1,6
11159,2011-01-07,723484101,0.003131,2011,1,7
...,...,...,...,...,...,...
3142674,2020-12-24,12504L109,0.002864,2020,12,24
3142675,2020-12-28,12504L109,0.000635,2020,12,28
3142676,2020-12-29,12504L109,-0.013751,2020,12,29
3142677,2020-12-30,12504L109,0.004658,2020,12,30


In [204]:
just_ret = just_ret.merge(FF_4_daily, on = ['Year', 'Month', 'Day'])

In [209]:
just_ret.drop(['Year','Month','Day', 'date'], axis=1, inplace =True)
just_ret = just_ret[just_ret['datadate']>='2017-01-01']

In [217]:
dates_list = list(just_ret['datadate'].unique())

In [240]:
i

251

In [242]:
just_ret[(just_ret['datadate']>=dates_list[i-251]) & (just_ret['datadate']<dates_list[i])  ]

,datadate,cusip,ret,mktrf,smb,hml,rf,umd
557190,2017-01-03,723484101,-0.009141,0.0083,-0.0013,0.0005,0.00002,-0.0061
557191,2017-01-03,002824100,0.016525,0.0083,-0.0013,0.0005,0.00002,-0.0061
557192,2017-01-03,007903107,0.007905,0.0083,-0.0013,0.0005,0.00002,-0.0061
557193,2017-01-03,009158106,-0.002925,0.0083,-0.0013,0.0005,0.00002,-0.0061
557194,2017-01-03,011659109,-0.008261,0.0083,-0.0013,0.0005,0.00002,-0.0061
...,...,...,...,...,...,...,...,...
649804,2017-12-29,718172109,0.007983,-0.0057,-0.0031,0.0000,0.00004,-0.0018
649805,2017-12-29,00507V109,-0.001263,-0.0057,-0.0031,0.0000,0.00004,-0.0018
649806,2017-12-29,92345Y106,-0.004677,-0.0057,-0.0031,0.0000,0.00004,-0.0018
649807,2017-12-29,34959E109,-0.007298,-0.0057,-0.0031,0.0000,0.00004,-0.0018


In [262]:
for i in range(250, len(dates_list)):
#for i in range(251, 270):
    x = just_ret[(just_ret['datadate']>=dates_list[i-250]) & (just_ret['datadate']<dates_list[i])  ]
    df = (x.groupby(['cusip']).apply(ols_coef,'ret ~ 1 +  mktrf + smb + hml + umd'))
    df['datadate'] = dates_list[i] 
    df.reset_index(inplace = True)
    if i == 251:
        df_full = df.copy()
    else:
        df_full = pd.concat([df_full, df])

    

In [ ]:
df_full_2 = df_full.copy()

In [300]:
df_full_2['datadate'] = df_full_2.groupby('cusip')['datadate'].shift(1)

In [302]:
df_full_2.drop(['new_d'], axis=1, inplace =True)

In [265]:
just_ret_res =  pd.read_pickle('/largedisks/ajim/KDD21_data/Daily/Just_ret_result.pickle')
just_ret_res = just_ret_res.stack().reset_index()
just_ret_res.rename(columns={0: 'DY_Pred'}, inplace =True) 

In [314]:
just_ret_res

,datadate,cusip,DY_Pred
0,2018-02-23,1055102,-0.009992
1,2018-02-23,26874784,-0.011896
2,2018-02-23,31100100,-0.013605
3,2018-02-23,31162100,-0.013708
4,2018-02-23,32654105,-0.016276
...,...,...,...
265675,2020-12-31,G96629103,0.007070
265676,2020-12-31,H1467J104,0.004177
265677,2020-12-31,H2906T109,0.014392
265678,2020-12-31,H84989104,0.006262


In [318]:
Merge_Df = df_full_2.merge(just_ret_res, on = ['cusip', 'datadate'])

In [319]:
Merge_Df = Merge_Df.merge(just_ret[['datadate','cusip','ret']], on = ['cusip', 'datadate'])

In [307]:
Merge_Df.set_index(['cusip', 'datadate'], inplace = True)

In [325]:
Merge_Df['DY_Pred2'] = Merge_Df.groupby('cusip')['DY_Pred'].shift()

In [327]:
Merge_Df_2 = Merge_Df.dropna()

In [328]:
Merge_Df_2

,cusip,Intercept,mktrf,smb,hml,umd,datadate,DY_Pred,ret,DY_Pred2
369,1055102,0.000597,0.614774,-0.083285,0.511225,0.040471,2018-02-26,-0.002249,0.013198,-0.009992
370,26874784,-0.000647,0.591012,-0.140470,0.661835,0.172120,2018-02-26,-0.001422,-0.017804,-0.011896
371,31100100,0.000877,0.840758,0.055958,0.311463,0.175689,2018-02-26,-0.004051,0.021140,-0.013605
372,31162100,-0.000557,1.139790,-0.232161,-0.211707,-0.207498,2018-02-26,-0.006204,0.009756,-0.013708
373,32654105,-0.000758,1.030778,-0.126712,-0.347101,0.533205,2018-02-26,-0.003822,0.021782,-0.016276
...,...,...,...,...,...,...,...,...,...,...
265306,G96629103,-0.000360,0.827422,-0.102963,0.297832,0.186344,2020-12-30,0.009419,-0.007972,0.006172
265307,H1467J104,-0.000045,0.882236,0.048901,0.764422,0.060116,2020-12-30,0.007932,0.006803,0.010350
265308,H2906T109,0.000029,0.875233,-0.061042,0.094103,-0.012907,2020-12-30,0.004707,0.001172,0.002428
265309,H84989104,0.000113,1.024391,0.728764,0.506417,0.126662,2020-12-30,0.007185,0.013535,0.006020


In [324]:
Merge_Df[Merge_Df['cusip']==26874784]

,cusip,Intercept,mktrf,smb,hml,umd,datadate,DY_Pred,ret
1,26874784,-0.000576,0.623259,-0.165902,0.657169,0.161362,2018-02-23,-0.011896,0.001833
370,26874784,-0.000647,0.591012,-0.140470,0.661835,0.172120,2018-02-26,-0.001422,-0.017804
739,26874784,-0.000663,0.603622,-0.138341,0.665474,0.172584,2018-02-27,0.003043,-0.013306
1108,26874784,-0.000697,0.613768,-0.133640,0.667055,0.166048,2018-02-28,-0.009075,-0.015402
1477,26874784,-0.000731,0.619490,-0.142316,0.667131,0.167177,2018-03-01,-0.011903,-0.015289
...,...,...,...,...,...,...,...,...,...
263467,26874784,-0.001216,1.278959,0.518659,1.254974,0.041704,2020-12-23,-0.000908,0.017242
263836,26874784,-0.001207,1.278847,0.518426,1.255554,0.042071,2020-12-24,0.001871,-0.001871
264205,26874784,-0.001278,1.278622,0.525064,1.253785,0.042377,2020-12-28,0.011267,-0.008060
264574,26874784,-0.001242,1.277405,0.515640,1.255114,0.040859,2020-12-29,0.010039,0.002425


In [323]:
Merge_Df

,cusip,Intercept,mktrf,smb,hml,umd,datadate,DY_Pred,ret
0,1055102,0.000581,0.607337,-0.077215,0.512246,0.042923,2018-02-23,-0.009992,0.011679
1,26874784,-0.000576,0.623259,-0.165902,0.657169,0.161362,2018-02-23,-0.011896,0.001833
2,31100100,0.000815,0.828636,0.088123,0.307132,0.176494,2018-02-23,-0.013605,0.003785
3,31162100,-0.000527,1.142673,-0.249760,-0.207999,-0.206264,2018-02-23,-0.013708,0.022866
4,32654105,-0.000792,1.018668,-0.112488,-0.346607,0.536576,2018-02-23,-0.016276,0.020232
...,...,...,...,...,...,...,...,...,...
265306,G96629103,-0.000360,0.827422,-0.102963,0.297832,0.186344,2020-12-30,0.009419,-0.007972
265307,H1467J104,-0.000045,0.882236,0.048901,0.764422,0.060116,2020-12-30,0.007932,0.006803
265308,H2906T109,0.000029,0.875233,-0.061042,0.094103,-0.012907,2020-12-30,0.004707,0.001172
265309,H84989104,0.000113,1.024391,0.728764,0.506417,0.126662,2020-12-30,0.007185,0.013535


In [309]:
exog = sm.add_constant(Merge_Df[['mktrf', 'smb','hml','umd', 'DY_Pred']])
endog = Merge_Df[['ret']]

In [312]:
exog = sm.add_constant(Merge_Df_2[['mktrf', 'DY_Pred2']])
endog = Merge_Df[['ret']]

In [331]:
exog = sm.add_constant(Merge_Df_2[['mktrf', 'smb','hml','umd', 'DY_Pred2']])
endog = Merge_Df_2[['ret']]

In [338]:
exog = sm.add_constant(Merge_Df_2[['mktrf', 'smb','hml', 'umd', 'DY_Pred2']])
endog = Merge_Df_2[['ret']]

In [340]:
mod = sm.OLS(endog, exog)
results = mod.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     29.99
Date:                Thu, 06 Oct 2022   Prob (F-statistic):           1.39e-30
Time:                        09:00:56   Log-Likelihood:             5.9684e+05
No. Observations:              264942   AIC:                        -1.194e+06
Df Residuals:                  264936   BIC:                        -1.194e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0009      0.000      5.134      0.000       0.001       0.001
mktrf         -0.0007      0.000     -4.048      0.000      -0.001      -0.000
smb            0.0005      0.000      3.245      0.001       0.000       0.001
hml           -0.0004   9.93e-05     -4.369      0.000      -0.001      -0.000
umd           -0.0003      0.000     -2.866      0.004      -0.001      -0.000
DY_Pred2       0.0527      0.005      9.829      0.000       0.042       0.063
==============================================================================
Omnibus:                   298798.156   Durbin-Watson:                   1.200
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        813378954.012
Skew:                          -4.705   Prob(JB):                         0.00
Kurtosis:                     274.279   Cond. No.                         153.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [388]:
Merge_Df_2

,cusip,Intercept,mktrf,smb,hml,umd,datadate,DY_Pred,ret,DY_Pred2,intercept
369,1055102,0.000597,0.614774,-0.083285,0.511225,0.040471,2018-02-26,-0.002249,0.013198,-0.009992,1
370,26874784,-0.000647,0.591012,-0.140470,0.661835,0.172120,2018-02-26,-0.001422,-0.017804,-0.011896,1
371,31100100,0.000877,0.840758,0.055958,0.311463,0.175689,2018-02-26,-0.004051,0.021140,-0.013605,1
372,31162100,-0.000557,1.139790,-0.232161,-0.211707,-0.207498,2018-02-26,-0.006204,0.009756,-0.013708,1
373,32654105,-0.000758,1.030778,-0.126712,-0.347101,0.533205,2018-02-26,-0.003822,0.021782,-0.016276,1
...,...,...,...,...,...,...,...,...,...,...,...
265306,G96629103,-0.000360,0.827422,-0.102963,0.297832,0.186344,2020-12-30,0.009419,-0.007972,0.006172,1
265307,H1467J104,-0.000045,0.882236,0.048901,0.764422,0.060116,2020-12-30,0.007932,0.006803,0.010350,1
265308,H2906T109,0.000029,0.875233,-0.061042,0.094103,-0.012907,2020-12-30,0.004707,0.001172,0.002428,1
265309,H84989104,0.000113,1.024391,0.728764,0.506417,0.126662,2020-12-30,0.007185,0.013535,0.006020,1


In [389]:
# centrality breakdown
grp_cen = Merge_Df_2.groupby(['datadate'])['DY_Pred2'].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]).reset_index()

In [390]:
grp_cen = grp_cen[["datadate", '10%', '20%', '30%', '40%','50%', '60%', '70%', '80%', '90%']]

grp_cen = grp_cen.rename(columns={'10%':'cn10','20%':'cn20','30%':'cn30','40%':'cn40',
                       '50%':'cn50','60%':'cn60','70%':'cn70', '80%':'cn80',
                        '90%':'cn90'})

In [384]:
centrality_grp_daily =  Merge_Df_2[['cusip','datadate', 'ret', 'DY_Pred2', 'DY_Pred']].merge(grp_cen, how='left', on=['datadate'])

In [391]:
centrality_grp_daily['cn_port'] = centrality_grp_daily.apply(cn_bucket, axis=1)

In [394]:
centrality_grp_daily

,cusip,datadate,ret,DY_Pred2,DY_Pred,cn10,cn20,cn30,cn40,cn50,cn60,cn70,cn80,cn90,cn_port
0,1055102,2018-02-26,0.013198,-0.009992,-0.002249,-0.015933,-0.015164,-0.014573,-0.014179,-0.013878,-0.013530,-0.012983,-0.011879,-0.009862,CD100
1,26874784,2018-02-26,-0.017804,-0.011896,-0.001422,-0.015933,-0.015164,-0.014573,-0.014179,-0.013878,-0.013530,-0.012983,-0.011879,-0.009862,CD100
2,31100100,2018-02-26,0.021140,-0.013605,-0.004051,-0.015933,-0.015164,-0.014573,-0.014179,-0.013878,-0.013530,-0.012983,-0.011879,-0.009862,CD100
3,31162100,2018-02-26,0.009756,-0.013708,-0.006204,-0.015933,-0.015164,-0.014573,-0.014179,-0.013878,-0.013530,-0.012983,-0.011879,-0.009862,CD100
4,32654105,2018-02-26,0.021782,-0.016276,-0.003822,-0.015933,-0.015164,-0.014573,-0.014179,-0.013878,-0.013530,-0.012983,-0.011879,-0.009862,CD100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264937,G96629103,2020-12-30,-0.007972,0.006172,0.009419,-0.022766,-0.009659,0.000454,0.005342,0.006513,0.007724,0.009645,0.010828,0.011657,CD70
264938,H1467J104,2020-12-30,0.006803,0.010350,0.007932,-0.022766,-0.009659,0.000454,0.005342,0.006513,0.007724,0.009645,0.010828,0.011657,CD70
264939,H2906T109,2020-12-30,0.001172,0.002428,0.004707,-0.022766,-0.009659,0.000454,0.005342,0.006513,0.007724,0.009645,0.010828,0.011657,CD40
264940,H84989104,2020-12-30,0.013535,0.006020,0.007185,-0.022766,-0.009659,0.000454,0.005342,0.006513,0.007724,0.009645,0.010828,0.011657,CD60


In [405]:
FF['ret_2'] = FF['ret']*252

In [401]:
FF = centrality_grp_daily.groupby(['datadate', 'cn_port'])['ret'].mean().reset_index()
FF.groupby(['cn_port'])['ret'].mean().reset_index()

,cn_port,ret
0,CD10,0.000632
1,CD100,-0.000371
2,CD20,0.000709
3,CD30,0.000734
4,CD40,0.000966
5,CD50,0.001199
6,CD60,-0.000469
7,CD70,0.000693
8,CD80,0.000211
9,CD90,0.000222


In [406]:
FF.groupby(['cn_port'])['ret_2'].mean().reset_index()

,cn_port,ret_2
0,CD10,0.159234
1,CD100,-0.093565
2,CD20,0.178745
3,CD30,0.185066
4,CD40,0.243467
5,CD50,0.302098
6,CD60,-0.118063
7,CD70,0.174572
8,CD80,0.053050
9,CD90,0.055978


In [408]:
(FF.groupby(['cn_port'])['ret'].mean()/FF.groupby(['cn_port'])['ret'].sem()).reset_index()

,cn_port,ret
0,CD10,1.058410
1,CD100,-0.505768
2,CD20,1.253598
3,CD30,1.481764
4,CD40,1.535600
5,CD50,2.184959
6,CD60,-0.651704
7,CD70,1.158905
8,CD80,0.317528
9,CD90,0.361332


In [409]:
(FF.groupby(['cn_port'])['ret_2'].mean()/FF.groupby(['cn_port'])['ret_2'].sem()).reset_index()

,cn_port,ret_2
0,CD10,1.058410
1,CD100,-0.505768
2,CD20,1.253598
3,CD30,1.481764
4,CD40,1.535600
5,CD50,2.184959
6,CD60,-0.651704
7,CD70,1.158905
8,CD80,0.317528
9,CD90,0.361332


In [410]:
FF

,datadate,cn_port,ret,ret_2
0,2018-02-26,CD100,0.007520,1.895077
1,2018-02-26,CD50,0.002254,0.567888
2,2018-02-27,CD10,-0.014797,-3.728767
3,2018-02-27,CD100,-0.012802,-3.226096
4,2018-02-27,CD30,-0.022555,-5.683885
...,...,...,...,...
5919,2020-12-30,CD50,0.007549,1.902321
5920,2020-12-30,CD60,0.007255,1.828335
5921,2020-12-30,CD70,0.009842,2.480130
5922,2020-12-30,CD80,0.009680,2.439284


In [419]:
from scipy import stats

In [424]:
CR_10 = FF[(FF['cn_port']=='CD60')]
CR_100 = FF[(FF['cn_port']=='CD50')]
print(np.round(CR_100['ret_2'].mean()*100-CR_10['ret_2'].mean()*100, 4))

print(stats.ttest_ind(CR_100['ret_2'], CR_10['ret_2']))

42.0161
Ttest_indResult(statistic=1.8415016306686063, pvalue=0.06580842927336578)


42.0161

Ttest_indResult(statistic=1.8415016306686063, pvalue=0.06580842927336578)

In [433]:
# centrality breakdown
grp_cen = Merge_Df.groupby(['datadate'])['DY_Pred'].describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]).reset_index()

In [434]:
grp_cen = grp_cen[["datadate", '10%', '20%', '30%', '40%','50%', '60%', '70%', '80%', '90%']]

grp_cen = grp_cen.rename(columns={'10%':'cn10','20%':'cn20','30%':'cn30','40%':'cn40',
                       '50%':'cn50','60%':'cn60','70%':'cn70', '80%':'cn80',
                        '90%':'cn90'})

In [436]:
centrality_grp_daily =  Merge_Df[['cusip','datadate', 'ret', 'DY_Pred']].merge(grp_cen, how='left', on=['datadate'])

In [438]:
centrality_grp_daily['cn_port'] = centrality_grp_daily.apply(cn_bucket, axis=1)

In [441]:
centrality_grp_daily['ret_2'] = centrality_grp_daily['ret']*252*100

In [443]:
FF = centrality_grp_daily.groupby(['datadate', 'cn_port'])['ret_2'].mean().reset_index()
FF.groupby(['cn_port'])['ret_2'].mean().reset_index()

,cn_port,ret_2
0,CD10,-7.919300
1,CD100,0.392758
2,CD20,-1.260541
3,CD30,3.887515
4,CD40,5.727891
5,CD50,3.887488
6,CD60,10.906007
7,CD70,2.375933
8,CD80,10.804759
9,CD90,4.728559


In [445]:
FF = centrality_grp_daily.groupby(['datadate', 'cn_port'])['ret'].mean().reset_index()
FF.groupby(['cn_port'])['ret'].mean().reset_index()

,cn_port,ret
0,CD10,-0.000314
1,CD100,0.000016
2,CD20,-0.000050
3,CD30,0.000154
4,CD40,0.000227
5,CD50,0.000154
6,CD60,0.000433
7,CD70,0.000094
8,CD80,0.000429
9,CD90,0.000188


In [446]:
(FF.groupby(['cn_port'])['ret'].mean()/FF.groupby(['cn_port'])['ret'].sem()).reset_index()

,cn_port,ret
0,CD10,-0.524126
1,CD100,0.024171
2,CD20,-0.085111
3,CD30,0.253057
4,CD40,0.374604
5,CD50,0.247719
6,CD60,0.701437
7,CD70,0.148883
8,CD80,0.660866
9,CD90,0.289468


In [444]:
(FF.groupby(['cn_port'])['ret_2'].mean()/FF.groupby(['cn_port'])['ret_2'].sem()).reset_index()

,cn_port,ret_2
0,CD10,-0.524126
1,CD100,0.024171
2,CD20,-0.085111
3,CD30,0.253057
4,CD40,0.374604
5,CD50,0.247719
6,CD60,0.701437
7,CD70,0.148883
8,CD80,0.660866
9,CD90,0.289468


In [333]:
result = fama_macbeth(Merge_Df_2,'datadate','ret',['mktrf','smb','hml', 'umd', 'DY_Pred2'],intercept=True)
result.head()
fm_summary(result)

/home/ajim/miniconda3/lib/python3.9/site-packages/finance_byu/fama_macbeth.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['intercept'] = 1


,mean,std_error,tstat
intercept,0.000598,0.000442,1.354158
mktrf,-0.000148,0.000674,-0.219966
smb,0.000031,0.000357,0.085916
hml,-0.000722,0.000443,-1.628886
umd,-0.000004,0.000511,-0.007728
DY_Pred2,-0.005515,0.041664,-0.132363


In [311]:
result = fama_macbeth(Merge_Df,'datadate','ret',['mktrf','smb','hml', 'umd', 'DY_Pred'],intercept=True)
result.head()
fm_summary(result)

,mean,std_error,tstat
intercept,0.000444,0.000448,0.989358
mktrf,-0.000127,0.000676,-0.187923
smb,0.000089,0.000359,0.247460
hml,-0.000690,0.000434,-1.589454
umd,-0.000082,0.000503,-0.162439
DY_Pred,-0.001852,0.036645,-0.050536


In [273]:
Merge_Df

,cusip,Intercept,mktrf,smb,hml,umd,datadate,DY_Pred,ret
0,1055102,0.000610,0.603394,-0.060072,0.506607,0.043274,2018-02-23,-0.009992,0.011679
1,26874784,-0.000522,0.636503,-0.159288,0.647693,0.154462,2018-02-23,-0.011896,0.001833
2,31100100,0.000823,0.845649,0.070453,0.306543,0.169963,2018-02-23,-0.013605,0.003785
3,31162100,-0.000506,1.134159,-0.230671,-0.212240,-0.203908,2018-02-23,-0.013708,0.022866
4,32654105,-0.000785,1.010900,-0.100031,-0.348280,0.539167,2018-02-23,-0.016276,0.020232
...,...,...,...,...,...,...,...,...,...
265675,G96629103,-0.000360,0.827422,-0.102963,0.297832,0.186344,2020-12-31,0.007070,0.021785
265676,H1467J104,-0.000045,0.882236,0.048901,0.764422,0.060116,2020-12-31,0.004177,0.013013
265677,H2906T109,0.000029,0.875233,-0.061042,0.094103,-0.012907,2020-12-31,0.014392,0.001171
265678,H84989104,0.000113,1.024391,0.728764,0.506417,0.126662,2020-12-31,0.006262,0.010879


In [130]:
#!pip install finance-byu
from finance_byu.fama_macbeth import fama_macbeth, fama_macbeth_parallel, fm_summary, fama_macbeth_numba
import time

In [131]:
n_jobs = 5
n_firms = 1.0e2
n_periods = 1.0e2

In [135]:
def firm(fid):
    f = np.random.random((int(n_periods),4))
    f = pd.DataFrame(f)
    f['period'] = f.index
    f['firmid'] = fid
    return f

In [136]:
df = [firm(i) for i in range(int(n_firms))]
df = pd.concat(df).rename(columns={0:'ret',1:'exmkt',2:'smb',3:'hml'})
df.head()

,ret,exmkt,smb,hml,period,firmid
0,0.272172,0.129020,0.695642,0.000678,0,0
1,0.452509,0.493258,0.768704,0.490502,1,0
2,0.177633,0.776947,0.353140,0.212524,2,0
3,0.956027,0.903125,0.038830,0.299062,3,0
4,0.354913,0.916090,0.773574,0.428018,4,0


In [124]:
cus = list(just_ret_res['cusip'].unique())

In [128]:
for i in cus:
    print(i)

1055102
26874784
31100100
31162100
32654105
37411105
37833100
38222105
53015103
00130H105
002824100
00507V109
00751Y106
007903107
009158106
00971T101
011659109
012653101
015271109
016255101
020002101
02209S103
023135106
023608102
025537101
025816109
03027X100
030420103
03073E105
03076C106
032095101
03662Q105
036752103
03748R747
04621X108
049560105
053484101
058498106
064058100
071813109
075887109
084423102
090572207
09247X101
093671105
097023105
09857L108
099724106
101121101
101137107
110122108
11133T103
12504L109
125269100
12541W209
125523100
12572Q105
125896100
126650100
127097103
127387108
134429109
14040H105
14149Y108
143130102
149123101
150870103
15135B101
15189T107
156782104
16119P108
166764100
169656105
171340102
172062101
172908105
172967424
177376100
189054109
191216100
192446102
194162103
200340107
20605P101
20825C104
209115104
21036P108
217204106
219350105
22160K105
22822V101
231021106
23331A109
233331107
235851102
237194105
23918K108
244199105
247361702
25179M103
253868103


In [37]:
mod = sm.OLS(endog, exog)
results = mod.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 ex_ret   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     20.42
Date:                Wed, 05 Oct 2022   Prob (F-statistic):           2.04e-20
Time:                        19:33:08   Log-Likelihood:                 30829.
No. Observations:               31893   AIC:                        -6.165e+04
Df Residuals:                   31887   BIC:                        -6.160e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.001      6.213      0.000       0.005       0.009
b_mkt         -0.0008      0.001     -0.773      0.439      -0.003       0.001
b_smb         -0.0005      0.001     -0.649      0.516      -0.002       0.001
b_hml          0.0001      0.001      0.146      0.884      -0.002       0.002
b_umd         -0.0041      0.001     -3.742      0.000      -0.006      -0.002
DY_Pred       -0.1444      0.015     -9.477      0.000      -0.174      -0.115
==============================================================================
Omnibus:                    18190.152   Durbin-Watson:                   1.610
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2136869.215
Skew:                           1.795   Prob(JB):                         0.00
Kurtosis:                      42.939   Cond. No.                         45.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
from linearmodels import PanelOLS
from linearmodels.panel import PooledOLS

In [86]:
mod = PooledOLS( endog, exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                 ex_ret   R-squared:                        0.0027
Estimator:                  PooledOLS   R-squared (Between):              0.0157
No. Observations:               31893   R-squared (Within):               0.0026
Date:                Wed, Oct 05 2022   R-squared (Overall):              0.0027
Time:                        19:58:34   Log-likelihood                 3.082e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      43.505
Entities:                         443   P-value                           0.0000
Avg Obs:                       71.993   Distribution:                 F(2,31890)
Min Obs:                       69.000                                           
Max Obs:                       72.000   F-statistic (robust):             43.505
                            

In [69]:
FF4_Data_2.set_index(['ID', 'Year', 'Month'])

,,,PERMNO,b_mkt,b_smb,b_hml,b_umd,R2,exret,ex_ret,DY_Pred
ID,Year,Month,,,,,,,,,
0,2014,1,54594,1.3916,2.4843,-0.1326,0.0224,58.1503%,-0.030671,-0.045876,0.045970
1,2014,1,20482,0.9039,-1.1467,-0.0883,0.3494,50.2645%,-0.011727,-0.037829,0.020188
2,2014,1,75107,1.4633,1.6234,-0.1502,0.2847,21.0552%,0.059163,0.028048,0.035505
3,2014,1,60038,0.2437,0.0355,0.4348,0.0357,16.7561%,0.000059,-0.011409,0.012921
4,2014,1,75510,1.1909,0.0202,0.2098,-0.3208,59.3198%,0.037462,-0.011507,0.031787
...,...,...,...,...,...,...,...,...,...,...,...
438,2019,12,83011,0.8048,1.3363,-0.0743,0.3335,38.6562%,0.032771,0.056813,0.011242
439,2019,12,51086,1.4562,0.9316,1.5484,-0.0612,39.1199%,0.040720,0.113264,0.008415
440,2019,12,42286,0.7293,1.0017,-0.4994,-0.4125,34.0786%,0.027733,0.052806,-0.005787


In [ ]:
def generate_graph_seq2seq_io_data(
        df, x_offsets, y_offsets, add_time_in_day=True, add_day_in_week=False, scaler=None
):
    """
    Generate samples from
    :param df:
    :param x_offsets:
    :param y_offsets:
    :param add_time_in_day:
    :param add_day_in_week:
    :param scaler:
    :return:
    # x: (epoch_size, input_length, num_nodes, input_dim)
    # y: (epoch_size, output_length, num_nodes, output_dim)
    """

    num_samples, num_nodes = df.shape
    data = np.expand_dims(df.values, axis=-1)
    data_list = [data]
    if add_time_in_day:
        time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
        time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
        data_list.append(time_in_day)
    if add_day_in_week:
        day_in_week = np.zeros(shape=(num_samples, num_nodes, 7))
        day_in_week[np.arange(num_samples), :, df.index.dayofweek] = 1
        data_list.append(day_in_week)

    data = np.concatenate(data_list, axis=-1)
    # epoch_len = num_samples + min(x_offsets) - max(y_offsets)
    x, y = [], []
    # t is the index of the last observation.
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = data[t + x_offsets, ...]
        y_t = data[t + y_offsets, ...]
        x.append(x_t)
        y.append(y_t)
    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)
    return x, y



def plot_loss(dic):
    plt.figure(figsize=(12,5))
    #plt.subplot(1, 2, 1)
    plt.plot(dic['loss'], label='Training loss ')
    plt.plot(dic['val_loss'], label='Validation loss ')
    plt.legend(frameon=False)
    plt.title('train and validation loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()


    
def MSE(original, imputed):
    # calculate Mean Squared Error
    return np.square(original-imputed).mean()

def MPE(y_true, y_pred, threshold=0.1):
    v = np.copy(y_true)
    np.place(v, v==0, threshold)
    #v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return np.mean(diff, axis=-1).mean()

"""
def R_squared(original, predicted):
    Differ = np.square(original-predicted)
    m = np.mean(original) 
    denom = np.square(original - m)
    R_sq = 1 - ((Differ.sum())/denom.sum())
    return R_sq
"""
def R_squared(original, predicted):
    Differ = np.square(original-predicted)
    #m = np.mean(original) 
    denom = np.square(original)
    R_sq = 1 - ((Differ.sum())/denom.sum())
    return R_sq

def Get_performance(Y_true, Y_pred): 
    print ("R2 : ", np.round(R_squared(Y_true, Y_pred), 4))
    print ("MSE",  np.round(MSE(Y_true, Y_pred), 4))
    print ("MPE", np.round(MPE(Y_true, Y_pred), 4))
    
    return (np.round(R_squared(Y_true, Y_pred), 4), np.round(MSE(Y_true, Y_pred), 4), np.round(MPE(Y_true, Y_pred), 4)) 
        

def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return 100.0 * np.mean(diff, axis=-1).mean()

def get_metrics(y, yp):
    return {
        "rmse": np.round(rmse(y, yp), 4),
        "mae": np.round(mae(y, yp), 4),
        "mape": np.round(mape(y, yp),4),
        #"MPE": np.round(MPE(y, yp), 4),
        #"R2": np.round(R_squared(y, yp),4)
        
    }



def Hill_estimator(data):
    """
    Returns the Hill Estimators for some 1D data set.
    """    
    # sort data in such way that the smallest value is first and the largest value comes last:
    Y = np.sort(data)
    n = len(Y)

    Hill_est = np.zeros(n-1)

    for k in range(0, n-1):    # k = 0,...,n-2
        summ = 0

        for i in range(0,k+1):   # i = 0, ..., k
            summ += np.log(Y[n-1-i]) - np.log(Y[n-2-k])
        
        Hill_est[k] = (1 / (k+1)) * summ      # add 1 to k because of Python syntax
  
    kappa = 1. / Hill_est
    return kappa

In [10]:
n_month = len(CRSP_data.groupby('date').size())

In [11]:
CRSP_data_2 = CRSP_data[['date','ID', 'me', 'm_1', 'm_6', 'm_12',
                         'size', 'c_SHROUT', 'turnover', 'grvol', 'Rel2High', 'Rel2low', 'b_mkt',
                         'b_smb', 'b_hml', 'ivol', 'tvol', 'RF', 'ex_ret',]]
CRSP_data_2.set_index(['date','ID'], inplace = True)

CRSP_data_2.clip(lower=CRSP_data_2.quantile(0.01), upper=CRSP_data_2.quantile(0.99), axis=1, inplace = True)
CRSP_data_2.replace([np.inf, -np.inf], 0, inplace = True)

CRSP_data_2.sort_values(by = ['date', 'ID'], inplace = True)

/tmp/ipykernel_157304/1027774087.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP_data_2.clip(lower=CRSP_data_2.quantile(0.01), upper=CRSP_data_2.quantile(0.99), axis=1, inplace = True)
/tmp/ipykernel_157304/1027774087.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP_data_2.replace([np.inf, -np.inf], 0, inplace = True)
/tmp/ipykernel_157304/1027774087.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CRSP_data_2.sort_

In [12]:
CRSP_data_2.reset_index(inplace=True)

In [13]:
CRSP_data_2['ex_ret'].describe()

count    159480.000000
mean          0.010381
std           0.096370
min          -0.253118
25%          -0.042941
50%           0.007046
75%           0.059436
max           0.332914
Name: ex_ret, dtype: float64

In [14]:
n_firms = len(CRSP_data_2.groupby('ID').size())

In [15]:
CRSP_data_2.set_index(['date','ID'], inplace =True)

In [16]:
ret_data = CRSP_data_2[ "ex_ret"].to_frame().pivot_table(values='ex_ret', index='date', columns='ID', aggfunc='mean')

In [17]:
ret_data

ID,0,1,2,3,4,5,6,7,8,9,...,433,434,435,436,437,438,439,440,441,442
date,,,,,,,,,,,,,,,,,,,,,
1990-01-31,-0.127367,-0.044671,-0.176155,0.024603,0.157263,-0.085065,0.158479,-0.212049,0.027027,-0.085170,...,-0.112487,-0.081458,-0.081172,-0.000961,-0.138386,-0.059271,-0.030700,-0.072367,-0.087222,-0.068763
1990-02-28,-0.017605,-0.019162,-0.005700,0.082535,0.281534,0.132231,-0.018521,0.034300,0.047241,0.008688,...,-0.005700,0.044300,-0.179169,-0.222681,-0.069879,0.060338,0.122505,0.018110,-0.035286,0.051992
1990-03-30,-0.042545,0.018941,0.116888,0.007114,0.251782,0.099661,-0.112894,-0.111323,0.049466,-0.079450,...,0.055138,-0.041833,-0.036647,-0.024593,0.033923,0.082096,-0.074582,0.183367,0.055307,-0.002400
1990-04-30,-0.253118,0.014697,-0.080071,0.033100,-0.000321,-0.047996,-0.253118,0.167013,-0.084149,0.000792,...,-0.048446,-0.088533,0.005921,0.011619,-0.076667,0.041880,-0.055680,-0.144155,-0.024241,0.075669
1990-05-31,-0.018592,0.114923,0.243200,0.052687,-0.075427,0.193200,0.136057,0.030237,0.050671,0.039002,...,0.110547,0.060756,0.195732,0.138655,0.071533,0.186998,-0.083723,0.061382,0.141435,-0.006800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-30,0.024678,-0.022036,-0.253118,-0.026119,-0.049616,0.031241,0.199147,-0.224718,-0.053525,-0.045442,...,0.033819,-0.037275,-0.023793,-0.041289,-0.043808,-0.014797,-0.207059,-0.045799,-0.138845,-0.082363
2019-09-30,-0.042536,-0.021139,-0.080424,-0.009060,-0.030832,-0.080019,-0.145273,0.313079,0.085106,0.096959,...,0.063955,0.136743,-0.014384,-0.015165,0.033793,0.031331,0.199324,0.091071,0.044021,0.081675
2019-10-31,0.013424,0.001607,0.165401,0.030379,0.004581,0.168904,0.145114,0.311658,0.068135,-0.070045,...,0.053013,-0.040904,0.054713,0.056440,-0.074235,-0.011944,0.251955,0.048748,0.019582,0.087224


In [18]:
#how many time lag I wanna use
n_lag = 24
#how many time period I wanna predict
n_forward = 1

x_offsets = np.sort(np.arange(-n_lag+1, 1, 1))
y_offsets = np.sort(np.arange(1, n_forward+1, 1))

x, y = generate_graph_seq2seq_io_data(
    ret_data,
    x_offsets=x_offsets,
    y_offsets=y_offsets,
    add_time_in_day=False,
    add_day_in_week=False,
    )

In [19]:
x = np.transpose(x, (0, 2, 1, 3))
y = np.transpose(y, (0, 2, 1, 3))

#reshape to made three dimensional tensor
x = x.reshape(x.shape[0],x.shape[1],x.shape[2] )
y = y.reshape(y.shape[0],y.shape[1],y.shape[2] )

#x = np.transpose(x, (0, 2, 1))
#y = np.transpose(y, (0, 2, 1))



x = x.astype('float32')
y = y.astype('float32')

In [20]:
#Do Standardization
#standardization
scaler = StandardScaler()
#scaler.fit(data)
data_stand = scaler.fit_transform(CRSP_data_2)
#data_stand = pd.DataFrame(data_stand, index=data.index,  columns=data.columns)  
features = data_stand.reshape(n_month, n_firms, CRSP_data_2.shape[1])
features = features[n_lag-1:-n_forward,:,:]

In [21]:
AdPos = np.empty([x.shape[0], x.shape[1], x.shape[1]])
AdNeg = np.empty([x.shape[0], x.shape[1], x.shape[1]])

for i in range(0, x.shape[0]):
    aa = np.corrcoef(x[i],rowvar=True)
    P_val =  np.abs((aa*np.sqrt(n_lag-2))/(np.sqrt(1-np.square(aa))))
    P_val  = np.nan_to_num(P_val, nan=0.0, posinf=0, neginf=0)
    ab = n_lag/2 - 1
    P_val = 2*special.btdtr(ab, ab, 0.5*(1 - abs(np.float64(aa))))
    P_val = np.nan_to_num(P_val, nan=0.0, posinf=0, neginf=0)
    P_val  = np.where(P_val>0.05, 0, 1)
    
    Adjac = aa*P_val
    #adj_matrix[i] = P_val 
    
    AdPos[i] =  np.where(Adjac > 0,1,0 )
    AdNeg[i] =  np.where(Adjac < 0,1,0 )

/tmp/ipykernel_157304/2133742962.py:6: RuntimeWarning: divide by zero encountered in true_divide
  P_val =  np.abs((aa*np.sqrt(n_lag-2))/(np.sqrt(1-np.square(aa))))


In [22]:
x_train = x[:216,:,:]
y_train = y[:216,:,:]

x_val = x[216:264,:,:]
y_val = y[216:264,:,:]


x_test = x[264:,:,:]
y_test = y[264:,:,:]


features_train  = features[:216,:,:]
features_val  =   features[216:264,:,:]
features_test  =  features[264:,:,:]


#adj_train = Adj[:216,:,:]
#adj_val = Adj[216:264,:,:]
#adj_test = Adj[264:,:,:]


#"""
adj_pos_train = AdPos[:216,:,:]
adj_pos_val = AdPos[216:264,:,:]
adj_pos_test = AdPos[264:,:,:]

adj_neg_train = AdNeg[:216,:,:]
adj_neg_val = AdNeg[216:264,:,:]
adj_neg_test = AdNeg[264:,:,:]
#"""

In [23]:
learning_rate = 0.001
batch_size = 16
epochs = 100
seed = 42
verbose = 1

In [24]:
verbose

1

In [25]:
tf.keras.backend.clear_session()

In [ ]:
## Define Model

inputs_ret = Input(shape=(x.shape[1], x.shape[2], ))
inputs_feat = Input(shape=(features.shape[1], features.shape[2], ))
inputs_adj_pos = Input(shape=(features.shape[1],features.shape[1], ))
inputs_adj_neg = Input(shape=(features.shape[1],features.shape[1], ))
#inputs_3 = Input(shape=(data_shape[1],k, ))

GAT_output_P, Att_weights_P =  GATConv(8, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj_pos])
Att_weights_P = tf.math.reduce_mean(Att_weights_P, axis=2)

GAT_output_P_2 =  GATConv(8, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=False, 
                        activation='relu', use_bias=True)([inputs_feat, inputs_adj_pos])

diff_out = DiffusionConv(8, K=6, activation='tanh')([inputs_feat, Att_weights_P])
#diff_out = ARMAConv(16, order=2, iterations=1, share_weights=False, gcn_activation='relu', dropout_rate=0.0, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None)


con_out_P_2 = Concatenate(axis=-1)([diff_out, GAT_output_P_2, GAT_output_P])
dense_P = Dense(8, activation='relu')(con_out_P_2)



GAT_output_N, Att_weights_N =  GATConv(8, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=True, 
                                   activation='relu', use_bias=False)([inputs_ret, inputs_adj_neg])
Att_weights_N = tf.math.reduce_mean(Att_weights_N, axis=2)

#con_out_N = Concatenate(axis=-1)([GAT_output_N, GAT_output_NF])

GAT_output_N_2 =  GATConv(8, attn_heads=4, concat_heads=True, dropout_rate=0.3, return_attn_coef=False, 
                        activation='relu', use_bias=True)([inputs_feat, inputs_adj_neg])

diff_out_N = DiffusionConv(8, K=6, activation='tanh')([inputs_feat, Att_weights_N])
con_out_N_2 = Concatenate(axis=-1)([diff_out_N, GAT_output_N_2, GAT_output_N])
dense_N = Dense(8, activation='relu')(con_out_N_2)


con_out = Concatenate(axis=-1)([dense_P, dense_N])
dense_out = Dense(8, activation='linear')(con_out)
dense_out = Dropout(0.2)(dense_out)
outputs = Dense(1)(dense_out)




In [ ]:
model = Model(inputs=[inputs_ret, inputs_feat, inputs_adj_pos, inputs_adj_neg], outputs=outputs)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=["mae", "mape"])
model.summary()


In [14]:
daily_data =  pd.read_pickle('/home/ajim/Desktop/Graph Asset Pricing/Daily/daily_clear_ret.pickle')

In [17]:
daily_data

,datadate,cusip,capgn,divd,eps,ret,size,log_vol,grvol,grshrout,...,m_m,m_2m,m_3m,m_6m,Max_y,Min_y,Rel2High,Rel2low,spread,sp_opclose
11155,2011-01-03,723484101,0.0,0.0,1.40,0.000724,22.229424,13.838691,1.073191,0.0,...,0.008717,-0.012934,0.005318,0.121345,42.37,34.50,-0.021005,0.202319,0.5800,-0.08
11156,2011-01-04,723484101,0.0,0.0,1.40,0.004330,22.233754,13.633325,-0.205365,0.0,...,0.015969,-0.003116,0.000000,0.096703,42.37,34.50,-0.016757,0.207536,0.4800,0.08
11157,2011-01-05,723484101,0.0,0.0,1.40,-0.006744,22.227010,13.154641,-0.478685,0.0,...,0.010934,-0.021043,-0.001208,0.081010,42.37,34.50,-0.023366,0.199420,0.3175,-0.16
11158,2011-01-06,723484101,0.0,0.0,1.40,0.001931,22.228942,14.486690,1.332049,0.0,...,0.022440,-0.016268,0.006776,0.083203,42.37,34.50,-0.021477,0.201739,0.4100,0.00
11159,2011-01-07,723484101,0.0,0.0,1.40,0.003131,22.232072,14.685083,0.198393,0.0,...,0.029773,-0.003600,0.007966,0.080584,42.37,34.50,-0.018409,0.205507,0.3900,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142674,2020-12-24,12504L109,0.0,0.0,3.21,0.002864,23.773024,12.883118,-0.993633,0.0,...,0.023636,0.322554,0.306449,0.379893,66.75,29.83,-0.057228,1.109621,0.8950,-0.06
3142675,2020-12-28,12504L109,0.0,0.0,3.21,0.000635,23.773660,13.668628,0.785509,0.0,...,0.027531,0.339112,0.285303,0.353029,66.75,29.83,-0.056629,1.110962,0.7300,-0.29
3142676,2020-12-29,12504L109,0.0,0.0,3.21,-0.013751,23.759908,13.461833,-0.206795,0.0,...,0.015904,0.346537,0.283665,0.317368,66.75,29.83,-0.069513,1.082132,1.6900,-1.00
3142677,2020-12-30,12504L109,0.0,0.0,3.21,0.004658,23.764567,13.368226,-0.093606,0.0,...,0.020399,0.195875,0.284056,0.330242,66.75,29.83,-0.065169,1.091854,0.8100,0.23
